In [1]:
import pandas as pd
import asyncio
import nest_asyncio
nest_asyncio.apply()
import json
import aiohttp
from understat import Understat

import random
def calculateWinner(home, away):
    HomeGoals = 0
    AwayGoals = 0
    
    def testShots(shots):
        Goals = 0
        
        for shot in shots:
            if random.random() <= shot:
                Goals += 1
        return Goals
    
    HomeGoals = testShots(home)
    AwayGoals = testShots(away)
    
    #This is all that changes from above
    #We now pass a simple string, rather than ask for a print out.
    if HomeGoals > AwayGoals:
        return("home")
    elif AwayGoals > HomeGoals:
        return("away")
    else:
        return("draw")
        
        
        #Run xG calculator 10000 times to test winner %
def calculateChance(team1, team2):
    home = 0;
    away = 0;
    draw = 0;
    
    for i in range(0,10000):
        matchWinner = calculateWinner(team1,team2)
        if matchWinner == "home":
            home +=1
        elif matchWinner == "away":
            away +=1
        else:
            draw +=1
    home = home/100
    away = away/100
    draw = draw/100
    #print("Over 10000 games, home wins {}%, away wins {}% and there is a draw in {}% of games.".format(home, away, draw))
    return home, away, draw


import asyncio
import json

import aiohttp

from understat import Understat

#Here enter the undersat match_id for the game
#https://understat.com/match/12010 -  this is the Spurs 3-0 Leicester game, so match_id is 12010

match_id = 13427
async def get_win_percent(match_id):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        players = await understat.get_match_shots(match_id)
        d = json.dumps(players)
        with open('data.json', 'w') as f:
            json.dump(players, f)
        home = pd.DataFrame.from_dict(players['h'], orient='columns')
        away = pd.DataFrame.from_dict(players['a'], orient='columns')
        fixtures = await understat.get_league_results("epl",2019)
        results = pd.DataFrame.from_dict(fixtures, orient='columns')
        homexg = home['xG'].tolist()
        awayxg = away['xG'].tolist()
        #print(home)
        #print(away)
        h_team = home.iloc[0]['h_team']
        a_team = away.iloc[0]['a_team']
        homexg = list(map(float, homexg))
        awayxg = list(map(float, awayxg))
        h,a, d = calculateChance(homexg, awayxg)
        print("Over 10000 games, {} wins {}%, {} wins {}% and there is a draw in {}% of games.".format(h_team,h,a_team, a, d))
loop = asyncio.get_event_loop()
#loop.run_until_complete(main())



In [2]:
async def get_table(season_id,league):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_league_results(league,season_id)
        results = pd.DataFrame.from_dict(fixtures, orient='columns')
        l = len(results)
        print(l)
        for i in range(0,l):
            matc_id = results.iloc[i]['id']
            players = await understat.get_match_shots(matc_id)
            home = pd.DataFrame.from_dict(players['h'], orient='columns')
            away = pd.DataFrame.from_dict(players['a'], orient='columns')
            homexg = home['xG'].tolist()
            awayxg = away['xG'].tolist()
            #print(homexg)
            #print(awayxg)
            homexg = list(map(float, homexg))
            awayxg = list(map(float, awayxg))
            h,a,d = calculateChance(homexg, awayxg)
            h_team = home.iloc[0]['h_team']
            a_team = away.iloc[0]['a_team']
            pts.iloc[1][h_team] = pts.iloc[1][h_team] + 1.0
            pts.iloc[1][a_team] = pts.iloc[1][a_team] + 1.0
            pts.iloc[0][h_team] = pts.iloc[0][h_team] + (h/100)*3
            pts.iloc[0][a_team] = pts.iloc[0][a_team] + (a/100)*3
            pts.iloc[0][h_team] = pts.iloc[0][h_team] + (d/100)*1
            pts.iloc[0][a_team] = pts.iloc[0][a_team] + (d/100)*1
loop = asyncio.get_event_loop()

In [3]:
#Enter any match id you want, 12000 is the match_id of this game - https://understat.com/match/12000
match_id = 12000
asyncio.run(get_win_percent(match_id))

Over 10000 games, Newcastle United wins 13.64%, Tottenham wins 63.87% and there is a draw in 22.49% of games.


In [4]:
pts = pd.DataFrame({'Juventus': [0], 'Atalanta':[0], 'Inter':[0], 'Lazio':[0], 'Roma':[0], 'AC Milan':[0], 'Napoli':[0], 'Sassuolo':[0], 'Verona':[0], 'Parma Calcio 1913':[0], 'Fiorentina':[0], 'Bologna':[0], 'Cagliari':[0], 'Sampdoria':[0], 'Udinese':[0], 'Torino':[0], 'Genoa':[0], 'Lecce':[0], 'Brescia':[0], 'SPAL 2013':[0]})
pts.loc[len(pts)] = 0
pts = pts.astype(float)
asyncio.run(get_table(2019,"serie a"))
table = pts.transpose()
table.columns = ['Expected points','Games played']
table = table.sort_values(['Expected points'], ascending=[False])
table

360


,Expected points,Games played
Atalanta,73.6246,36.0
Inter,73.3020,36.0
Juventus,68.7623,36.0
Roma,65.3592,36.0
Napoli,62.8755,36.0
AC Milan,60.1564,36.0
Lazio,58.7810,36.0
Bologna,50.6944,36.0
Fiorentina,50.3592,36.0
Verona,47.1674,36.0


In [5]:
pts = pd.DataFrame({'Liverpool': [0], 'Manchester City':[0], 'Chelsea':[0], 'Leicester':[0], 'Manchester United':[0], 'Tottenham':[0], 'Wolverhampton Wanderers':[0], 'Sheffield United':[0], 'Burnley':[0], 'Arsenal':[0], 'Southampton':[0], 'Everton':[0], 'Newcastle United':[0], 'Crystal Palace':[0], 'West Ham':[0], 'Brighton':[0], 'Watford':[0], 'Aston Villa':[0], 'Bournemouth':[0], 'Norwich':[0]})
pts.loc[len(pts)] = 0
pts = pts.astype(float)
asyncio.run(get_table(2019,"epl"))
table = pts.transpose()
table.columns = ['Expected points','Games played']
table = table.sort_values(['Expected points'], ascending=[False])
table

380


,Expected points,Games played
Manchester City,87.0164,38.0
Liverpool,74.3158,38.0
Chelsea,73.5333,38.0
Manchester United,70.8105,38.0
Wolverhampton Wanderers,63.3472,38.0
Leicester,61.2270,38.0
Southampton,57.2294,38.0
Everton,55.7267,38.0
Arsenal,50.1899,38.0
Tottenham,49.7986,38.0


In [6]:
pts = pd.DataFrame({'Bayern Munich': [0], 'Borussia Dortmund':[0], 'RasenBallsport Leipzig':[0], 'Borussia M.Gladbach':[0], 'Bayer Leverkusen':[0], 'Hoffenheim':[0], 'Wolfsburg':[0], 'Freiburg':[0], 'Eintracht Frankfurt':[0], 'Hertha Berlin':[0], 'Union Berlin':[0], 'Schalke 04':[0], 'Mainz 05':[0], 'FC Cologne':[0], 'Augsburg':[0], 'Werder Bremen':[0], 'Fortuna Duesseldorf':[0], 'Paderborn':[0]})
pts.loc[len(pts)] = 0
pts = pts.astype(float)
asyncio.run(get_table(2019,"bundesliga"))
table = pts.transpose()
table.columns = ['Expected points','Games played']
table = table.sort_values(['Expected points'], ascending=[False])
table

306


,Expected points,Games played
Bayern Munich,76.9747,34.0
RasenBallsport Leipzig,68.7466,34.0
Borussia M.Gladbach,62.7731,34.0
Borussia Dortmund,62.1987,34.0
Wolfsburg,55.5675,34.0
Bayer Leverkusen,54.0509,34.0
Eintracht Frankfurt,52.4405,34.0
Hoffenheim,46.7040,34.0
FC Cologne,44.0711,34.0
Augsburg,39.9750,34.0


In [7]:
pts = pd.DataFrame({'Real Madrid': [0], 'Barcelona':[0], 'Atletico Madrid':[0], 'Sevilla':[0], 'Villarreal':[0], 'Real Sociedad':[0], 'Granada':[0], 'Getafe':[0], 'Valencia':[0], 'Osasuna':[0], 'Athletic Club':[0], 'Levante':[0], 'Real Valladolid':[0], 'Eibar':[0], 'Real Betis':[0], 'Alaves':[0], 'Celta Vigo':[0], 'Leganes':[0], 'Mallorca':[0], 'Espanyol':[0]})
pts.loc[len(pts)] = 0
pts = pts.astype(float)
asyncio.run(get_table(2019,"la liga"))
table = pts.transpose()
table.columns = ['Expected points','Games played']
table = table.sort_values(['Expected points'], ascending=[False])
table

380


,Expected points,Games played
Real Madrid,77.6798,38.0
Barcelona,73.2815,38.0
Atletico Madrid,72.5775,38.0
Sevilla,65.5500,38.0
Villarreal,63.8131,38.0
Real Sociedad,57.0663,38.0
Getafe,52.9770,38.0
Granada,52.8489,38.0
Osasuna,51.6254,38.0
Leganes,49.3816,38.0


In [8]:
pts = pd.DataFrame({'Paris Saint Germain': [0], 'Marseille':[0], 'Rennes':[0], 'Lille':[0], 'Reims':[0], 'Nice':[0], 'Lyon':[0], 'Montpellier':[0], 'Monaco':[0], 'Angers':[0], 'Strasbourg':[0], 'Bordeaux':[0], 'Nantes':[0], 'Brest':[0], 'Metz':[0], 'Dijon':[0], 'Saint-Etienne':[0], 'Nimes':[0], 'Amiens':[0], 'Toulouse':[0]})
pts.loc[len(pts)] = 0
pts = pts.astype(float)
asyncio.run(get_table(2019,"ligue 1"))
table = pts.transpose()
table.columns = ['Expected points','Games played']
table = table.sort_values(['Expected points'], ascending=[False])
table

279


,Expected points,Games played
Paris Saint Germain,65.7729,27.0
Lille,47.8114,28.0
Lyon,46.2682,28.0
Monaco,42.8116,28.0
Marseille,41.6525,28.0
Rennes,41.3054,28.0
Reims,40.0680,28.0
Montpellier,40.0108,28.0
Angers,37.9822,28.0
Strasbourg,37.7217,27.0
